In [ ]:
import os
import keras
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.optimizers import Adam, Nadam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
import os
import sys
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


%matplotlib inline

import cv2
from sklearn.model_selection import train_test_split

from tqdm import tqdm_notebook #, tnrange
from itertools import chain
from skimage.io import imread, imshow #, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

import tensorflow as tf

from keras.models import Model, load_model, save_model
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras import optimizers

import tensorflow as tf

from keras.preprocessing.image import array_to_img, img_to_array, load_img#,save_img

import time
t_start = time.time()

In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.7)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [ ]:
# Set some parameters
im_width = 256
im_height = 256
border = 5
path_train = "C:/Users/yousu/Downloads/BlastsOnline"

In [ ]:
# Get and resize train images and masks
def get_data(path, train=True):
    ids = next(os.walk(path + "/imagesResized_256_1368_updated"))[2]
    ids2= next(os.walk(path + "/labelResized_256_1368_updated"))[2]
    X = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)
    if train:
        y = np.zeros((len(ids2), im_height, im_width, 1), dtype=np.float32)
    print('Getting and resizing images ... ')
    for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
        # Load images
        img = load_img(path + '/imagesResized_256_1368_updated/' + id_, grayscale=True)
        x_img = img_to_array(img)
        x_img = resize(x_img, (im_width, im_height, 1), mode='constant', preserve_range=True)
        X[n] = x_img / 255

    for n, id_ in tqdm_notebook(enumerate(ids2), total=len(ids2)):
        # Load masks
        if train:
            mask = img_to_array(load_img(path + '/labelResized_256_1368_updated/' + id_, grayscale=True))
            mask = resize(mask, (im_width, im_height, 1), mode='constant', preserve_range=True)
            y[n] = mask / 255

        # Save images
        #X[n] = x_img / 255
        #if train:
            #y[n] = mask / 255
    print('Done!')
    if train:
        return X, y
    else:
        return X
    
X, y = get_data(path_train, train=True)

In [ ]:
first_image = y[114]
first_image = np.array(first_image)
pixels = first_image.reshape((im_width, im_height))
plt.imshow(pixels, cmap='gray')
plt.show()

In [ ]:
# Split train and valid
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=2018)

In [ ]:
first_image = X_train[58]
first_image = np.array(first_image)
pixels = first_image.reshape((im_width, im_height))
plt.imshow(pixels, cmap='gray')
plt.show()
print(len(y_valid))

In [ ]:
# Check if training data looks all right
ix = random.randint(0, len(X_train))
has_mask = y_train[ix].max() > 0

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

ax[0].imshow(X_train[ix,...,0], cmap='gray', interpolation='bilinear')
if has_mask:
    ax[0].contour(y_train[ix].squeeze(), colors='k', levels=[0.5])
ax[0].set_title('Embryo')

ax[1].imshow(y_train[ix].squeeze(), interpolation='bilinear', cmap='gray')
ax[1].set_title('Label');
#print(ix)

In [ ]:
from segmentation_models import Unet
#from segmentation_models.backbones import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    #Third Layer
    #x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               #padding="same")(x)
    #if batchnorm:
        #x = BatchNormalization()(x)
    #x = Activation("relu")(x)
    return x

In [ ]:
def get_unet(input_img, n_filters=16, dropout=0.05, batchnorm=True):
    # contracting path
    c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    #c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    #u6 = BatchNormalization()(u6) # added by me
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (2, 2), strides=(2, 2), padding='same') (c4)
    u7 = concatenate([u7, c3])
    #u7 = BatchNormalization()(u7) 
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    #u8 = BatchNormalization()(u8)
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    #u9 = BatchNormalization()(u9)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [ ]:
input_img = Input((im_height, im_width, 1), name='img')
model = get_unet(input_img, n_filters=16, dropout=0.05, batchnorm=True)
lr=0.0001
model.compile(optimizer=Nadam(lr), loss=bce_jaccard_loss, metrics=[iou_score])
model.summary()

In [ ]:
def show_flipped_images(x):
    fig, ax = plt.subplots(1, 4, figsize=(20, 10))
    ax[0].imshow(x[:,:,0], cmap='gray')
    ax[0].set_title('original')
    ax[1].imshow(np.fliplr(x[:,:,0]), cmap='gray')
    ax[2].imshow(np.flipud(x[:,:,0]), cmap='gray')
    ax[3].imshow(np.fliplr(np.flipud(x[:,:,0])), cmap='gray')

In [ ]:
show_flipped_images(X_train[54])

In [ ]:
data_gen_args = dict(horizontal_flip=True,
                    vertical_flip=True,
                    rotation_range=270,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    zoom_range=0.1)

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

seed = 2018
bs = 16 #batchsize

image_generator = image_datagen.flow(X_train, seed=seed, batch_size=bs, shuffle=True)
mask_generator = mask_datagen.flow(y_train, seed=seed, batch_size=bs, shuffle=True)

# Just zip the two generators to get a generator that provides augmented images and masks at the same time
train_generator = zip(image_generator, mask_generator)

In [ ]:
NAME = "UNET_MOD01-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

In [ ]:
callbacks = [
    EarlyStopping(patience=15, verbose=1),
    ReduceLROnPlateau(factor=0.05, patience=5, min_lr=0.000001, verbose=1),
    ModelCheckpoint('model-Unetplus_01.h5', verbose=1, save_best_only=True, save_weights_only=True),
    tensorboard
]

In [ ]:
results = model.fit_generator(train_generator, steps_per_epoch=(len(X_train) // bs), epochs=100, callbacks=callbacks,
                              validation_data=(X_valid, y_valid))

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(results.history["loss"], label="loss")
plt.plot(results.history["val_loss"], label="val_loss")
plt.plot( np.argmin(results.history["val_loss"]), np.min(results.history["val_loss"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

In [ ]:
# Load best model
model.load_weights('model-Unetplus_01.h5')

In [ ]:
# Evaluate on validation set (this must be equals to the best log_loss)
model.evaluate(X_valid, y_valid, verbose=1)

In [ ]:
model.save("UNETmodel.h5")

In [ ]:
# Predict on train, val and test
preds_train = model.predict(X_train, verbose=1)
preds_val = model.predict(X_valid, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.float32)
preds_val_t = (preds_val > 0.5).astype(np.float32)

In [ ]:
import matplotlib.image as mpimg
img_path = 'C:/Users/yousu/Downloads/BlastsOnline/001_ChKrGPS1.png'
img=mpimg.imread(img_path)
plt.imshow(img)

In [ ]:
x = img_to_array(img)
x = resize(x, (256, 256, 1), mode='constant', preserve_range=True)
x = np.expand_dims(x, axis=0)
x = x / 255
x.shape

In [ ]:
# Predict on train, val and test
preds_val = model.predict(x, verbose=1)

# Threshold predictions
preds_val_t = (preds_val > 0.5).astype(np.float32)

In [ ]:
def plot_sample(X, y, preds, binary_preds, ix=None):
    if ix is None:
        ix = random.randint(0, len(X))

    has_mask = y[ix].max() > 0

    fig, ax = plt.subplots(1, 4, figsize=(20, 10))
    ax[0].imshow(X[ix, ..., 0], cmap='gray')
    if has_mask:
        ax[0].contour(y[ix].squeeze(), colors='r', levels=[0.7])
    ax[0].set_title('Embryo')
    ax[0].grid(False)

    ax[1].imshow(y[ix].squeeze(), interpolation='bilinear', cmap='gray')
    ax[1].set_title('Embryo_GT')
    ax[1].grid(False)

    ax[2].imshow(preds[ix].squeeze(), vmin=0, vmax=1)
    if has_mask:
        ax[2].contour(y[ix].squeeze(), colors='r', levels=[0.7])
    ax[2].set_title('Embryo Predicted')
    ax[2].grid(False)
    
    ax[3].imshow(binary_preds[ix].squeeze(), vmin=0, vmax=1)
    if has_mask:
        ax[3].contour(y[ix].squeeze(), colors='r', levels=[0.7])
    ax[3].set_title('Embryo Predicted binary');
    ax[3].grid(False)

In [ ]:
# Check if training data looks all right
plot_sample(X_train, y_train, preds_train, preds_train_t, ix=66)

In [ ]:
# Check if valid data looks all right
plot_sample(X_valid, y_valid, preds_val, preds_val_t, ix=34)

In [ ]:
import cv2
from PIL import Image
import imageio

for x in range (len(X_valid)):
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.imshow(X_valid[x, ...,0],cmap='gray')
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])
    fig.savefig('Original' + '_' + str(x+1) + '.png')
    
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.imshow(preds_val_t[x].squeeze(), vmin=0, vmax=1)
    ax.contour(y_valid[x].squeeze(), colors='r', levels=[0.8])
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])
    fig.savefig('Predicted' + '_' + str(x+1) + '.png')
    
    image1 = Image.open('Original' + '_' + str(x+1) + '.png')
    image2 = Image.open('Predicted' + '_' + str(x+1) + '.png')
    image1 = image1.resize((500,500), Image.NEAREST)
    image2 = image2.resize((500,500), Image.NEAREST)
    alphaBlended1 = Image.blend(image1, image2, alpha=0.7)
    imageio.imwrite('Overlay' + '_' + str(x+1) + '.png', alphaBlended1)

In [ ]:
from keras import backend as K

def accuracy(y_true, y_pred):
     #test accuracy
     y_pred_pos = K.round(K.clip(y_pred, 0, 1))
     y_pred_neg = 1 - y_pred_pos
 
 
     y_pos = K.round(K.clip(y_true, 0, 1))
     y_neg = 1 - y_pos
 
 
     tp = K.sum(y_pos * y_pred_pos)
     tn = K.sum(y_neg * y_pred_neg)
 
 
     fp = K.sum(y_neg * y_pred_pos)
     fn = K.sum(y_pos * y_pred_neg)
 
 
     numerator = (tp + tn)
     denominator = (tp+tn+fp+fn)
 
 
     return numerator / (denominator + K.epsilon())

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

# fit the model
#history = model.fit(Xtrain, ytrain, validation_split=0.3, epochs=10, verbose=0)

# evaluate the model
y_valid = tf.convert_to_tensor(y_valid, np.float32)
preds_val_t = tf.convert_to_tensor(preds_val_t, np.float32)
#preds_val = tf.convert_to_tensor(preds_val, np.float32)

In [ ]:
with tf.Session() as sess:
    print(sess.run(recall_m(y_valid,preds_val_t)))
    print(sess.run(precision_m(y_valid,preds_val_t)))
    print(sess.run(f1_m(y_valid,preds_val_t)))

In [ ]:
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = tf.layers.flatten(y_true)
    y_pred_f = tf.layers.flatten(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
def jaccard(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, np.float32)
    y_pred = tf.convert_to_tensor(y_pred, np.float32)
    epsilon = 1e-15
    #intersection = (y_pred * y_true).sum(dim=-2).sum(dim=-1)
    #union = y_true.sum(dim=-2).sum(dim=-1) + y_pred.sum(dim=-2).sum(dim=-1)
    y_true_f = tf.layers.flatten(y_true)
    y_pred_f = tf.layers.flatten(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f)+tf.reduce_sum(y_pred_f)

    return (tf.reduce_mean(intersection + epsilon)/ (union - intersection + epsilon))

In [ ]:
from keras import backend as K

def specificity(y_true, y_pred):
     #test accuracy
     y_pred_pos = K.round(K.clip(y_pred, 0, 1))
     y_pred_neg = 1 - y_pred_pos
 
 
     y_pos = K.round(K.clip(y_true, 0, 1))
     y_neg = 1 - y_pos
 
 
     tp = K.sum(y_pos * y_pred_pos)
     tn = K.sum(y_neg * y_pred_neg)
 
 
     fp = K.sum(y_neg * y_pred_pos)
     fn = K.sum(y_pos * y_pred_neg)
 
 
     numerator = tn
     denominator = (tn+fp)
 
 
     return numerator / (denominator + K.epsilon())

In [ ]:
with tf.Session() as sess:
    print(sess.run(dice_coef(y_valid,preds_val_t)))
    print(sess.run(jaccard(y_valid,preds_val_t)))
    print(sess.run(accuracy(y_valid,preds_val_t)))
    print(sess.run(specificity(y_valid, preds_val_t)))